In [24]:
import os
import io
import struct
from typing import List, Tuple, Optional
from enum import Enum
import pandas as pd

from blowfish import Blowfish

In [25]:
class PackEntryType(Enum):
  Nop = 0
  Folder = 1
  File = 2

In [26]:
def generate_final_blowfish_key(password: str, salt: bytes) -> bytes:
    """
    Reproduces the 'GenerateFinalBlowfishKey' logic in C#.
    """
    # 1) Limit key length to max of 56
    plain_key_length = min(len(password), 56)

    # 2) Convert password to ASCII bytes
    a_key = password.encode("ascii")

    # 3) Create a 56-byte base key buffer
    b_key = bytearray(56)
    
    # Copy salt into b_key
    # Equivalent to `Array.ConstrainedCopy(salt, 0, bKey, 0, salt.Length)`
    b_key[:len(salt)] = salt

    # 4) Generate the final blowfish key by XOR-ing
    #    the ASCII password bytes with the corresponding part of b_key.
    bf_key = bytearray(plain_key_length)
    for x in range(plain_key_length):
        bf_key[x] = a_key[x] ^ b_key[x]

    return bytes(bf_key)

In [27]:
def read_string_with_length(stream, byte_count: int) -> str:
    """
    Read 'byte_count' bytes, then decode using self._encoding,
    stopping at the first 0 (null terminator) if present.
    """
    buffer = stream.read(byte_count)

    # Find null terminator offset (if any)
    terminator_offset = byte_count
    for i in range(byte_count):
        if buffer[i] == 0:
            terminator_offset = i
            break

    return buffer[:terminator_offset].decode('ascii', errors='replace')

In [28]:
# data_pk2_path = os.path.join(os.getcwd(), 'data', 'Data.pk2')
media_pk2_path = os.path.join(os.getcwd(), 'data', 'Media.pk2')
# print(data_pk2_path)
print(media_pk2_path)

/Users/hodung/Desktop/Workspace/silkroad/pk2-extractor/LoadTextData/data/Media.pk2


In [29]:
pk2key = '169841'
salt = [0x03, 0xF8, 0xE4, 0x44, 0x88, 0x99, 0x3F, 0x64, 0xFE, 0x35]

In [30]:
key = generate_final_blowfish_key(pk2key, bytes(salt))
key

b'2\xce\xdd|\xbc\xa8'

In [31]:
blowfish = Blowfish()
blowfish.Initialize(key)

In [32]:
_media_file_stream = open(media_pk2_path, 'rb')

In [33]:
signature = _media_file_stream.read(30)
version =  struct.unpack('<i', _media_file_stream.read(4))[0]
encrypted = _media_file_stream.read(1)
encryption_checksum = _media_file_stream.read(16)
payload = _media_file_stream.read(205)


In [34]:
Header = {
    'signature': signature,
    'version': version,
    'encrypted': encrypted,
    'encryption_checksum': encryption_checksum,
    'payload': payload
}

In [35]:
blowfish_checksum_decoded = "Joymax Pak File"

In [36]:
if blowfish and encrypted == b'\x01':
    temp_checksum = blowfish.Encode(blowfish_checksum_decoded.encode('ascii'))
    if temp_checksum is None or temp_checksum[0] != encryption_checksum[0] or temp_checksum[1] != encryption_checksum[1] or temp_checksum[2] != encryption_checksum[2]:
        raise Exception('Failed to open JoymaxPackFile: The password or salt is wrong.')

In [37]:
def read_block_at(position: int):
    _media_file_stream.seek(position, io.SEEK_SET)
    buffer = _media_file_stream.read(128 * 20)
    if blowfish is not None:
      entry_buffer = io.BytesIO(blowfish.Decode(buffer))
    else:
      entry_buffer = io.BytesIO(buffer)

    entries = []
    for _ in range(20):
      entry = {
        "Type": PackEntryType(entry_buffer.read(1)[0]),
        "Name": read_string_with_length(entry_buffer, 89).rstrip('\0'),
        "CreationTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "ModifyTime": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataPosition": struct.unpack('<q', entry_buffer.read(8))[0],
        "DataSize": struct.unpack('<i', entry_buffer.read(4))[0],
        "NextBlock": struct.unpack('<q', entry_buffer.read(8))[0],
        "Payload": entry_buffer.read(2)
      }
      entries.append(entry)
      # print(entry)

    return {
      "Position": position,
      "Entries": entries
    }


def read_blocks_at(position: int):
    result = []

    block = read_block_at(position)
    result.append(block)

    if block["Entries"][19]["NextBlock"] > 0:
        result.extend(read_blocks_at(block["Entries"][19]["NextBlock"]))

    return result

In [38]:
Root = read_blocks_at(256)

In [39]:
blocks_in_memory = {
  "": Root
}

In [40]:
blocks_in_memory

{'': [{'Position': 256,
   'Entries': [{'Type': <PackEntryType.Folder: 1>,
     'Name': '.',
     'CreationTime': 133336205048819447,
     'ModifyTime': 133336205048819447,
     'DataPosition': 256,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'acobject',
     'CreationTime': 133336205048829450,
     'ModifyTime': 133336205048829450,
     'DataPosition': 2816,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'config',
     'CreationTime': 133336205049179438,
     'ModifyTime': 133336205049179438,
     'DataPosition': 35136,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
    {'Type': <PackEntryType.Folder: 1>,
     'Name': 'Effect',
     'CreationTime': 133336205049679437,
     'ModifyTime': 133336205049679437,
     'DataPosition': 41896,
     'DataSize': 0,
     'NextBlock': 0,
     'Payload': b'\x00\x00'},
   

In [41]:
def get_entry_buffer(file_path: str) -> Optional[dict]:
  parent_folder_path = os.path.dirname(file_path)
  file_name = os.path.basename(file_path)
  if parent_folder_path not in blocks_in_memory:
    paths = parent_folder_path.split(os.path.sep)
    blocks = blocks_in_memory[""]
    current_path = ""

    for sub_folder_name in paths:
        for block in blocks:
            entries = block["Entries"]
            entry = next((e for e in entries if e["Name"] == sub_folder_name and e["Type"] == PackEntryType.Folder), None)
            if entry is None:
                continue
            
            current_path = os.path.join(current_path, entry["Name"])
            if current_path in blocks_in_memory:
                blocks = blocks_in_memory[current_path]
                break
            
            blocks = read_blocks_at(entry["DataPosition"])
            blocks_in_memory[current_path] = blocks
            break
        
  root = blocks_in_memory[parent_folder_path]
  entry = None
  for entries in root:
    entry = next((x for x in entries["Entries"] if x["Name"] == file_name.lower()), None)
    if entry:
      break

  _media_file_stream.seek(entry["DataPosition"], io.SEEK_SET)
  # _media_file_stream.read(entry["DataSize"])
  buffer = io.BytesIO(_media_file_stream.read(entry["DataSize"])) 

  return buffer
   

def get_lines(file_path: str) -> List[Tuple[str, int]]:

  buffer = get_entry_buffer(file_path)

  text = buffer.read().decode('utf-16', errors="replace")
  lines = text.split('\r\n')
  return lines  

def convert_string(value):
    try:
        # First, try to convert to an integer
        return int(value)
    except ValueError:
        try:
            # If it fails, try to convert to a float
            return float(value)
        except ValueError:
            # If both conversions fail, return the original string
            return value


In [42]:
text_data_directory = os.path.join('server_dep', 'silkroad', 'textdata')
text_data_directory

'server_dep/silkroad/textdata'

## Load Text Data

### TextQuest_Speech&Name.txt

In [43]:
text_quest_speech_name_list_file = os.path.join(text_data_directory, 'TextQuest_Speech&Name.txt')
text_quest_speech_name_list_file

'server_dep/silkroad/textdata/TextQuest_Speech&Name.txt'

In [44]:
text_quest_speech_name_files = get_lines(text_quest_speech_name_list_file)
text_quest_speech_name_files

['TextQuest_Speech&Name_100.txt',
 'TextQuest_Speech&Name_1000.txt',
 'TextQuest_Speech&Name_1100.txt',
 'TextQuest_Speech&Name_1200.txt',
 'TextQuest_Speech&Name_1300.txt',
 'TextQuest_Speech&Name_1400.txt',
 'TextQuest_Speech&Name_1500.txt',
 'TextQuest_Speech&Name_1600.txt',
 'TextQuest_Speech&Name_1700.txt',
 'TextQuest_Speech&Name_1800.txt',
 'TextQuest_Speech&Name_1900.txt',
 'TextQuest_Speech&Name_200.txt',
 'TextQuest_Speech&Name_2000.txt',
 'TextQuest_Speech&Name_2100.txt',
 'TextQuest_Speech&Name_2200.txt',
 'TextQuest_Speech&Name_2300.txt',
 'TextQuest_Speech&Name_2400.txt',
 'TextQuest_Speech&Name_2500.txt',
 'TextQuest_Speech&Name_2600.txt',
 'TextQuest_Speech&Name_2700.txt',
 'TextQuest_Speech&Name_2800.txt',
 'TextQuest_Speech&Name_2900.txt',
 'TextQuest_Speech&Name_300.txt',
 'TextQuest_Speech&Name_3000.txt',
 'TextQuest_Speech&Name_3100.txt',
 'TextQuest_Speech&Name_3200.txt',
 'TextQuest_Speech&Name_3300.txt',
 'TextQuest_Speech&Name_3400.txt',
 'TextQuest_Speech&Name

In [45]:
df_text_quest_speech_name = pd.DataFrame(columns=range(17))

for text_quest_speech_name_file in text_quest_speech_name_files:
  file_name = os.path.join(text_data_directory, text_quest_speech_name_file)
  lines = get_lines(file_name)
  for idx, line in enumerate(lines):
    values = line.split('\t')
    if len(values) < 17:
      continue
    df_text_quest_speech_name.loc[len(df_text_quest_speech_name)] = values
    


In [46]:
df_text_quest_speech_name

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16
0,1,1,SN_NPC_CH_SMITH_BS,장안에서 무기를 구하려거든 주저하지 말고 여기로 오시게! 내가 직접 만든 것 말고도...,0,0,0,0,0,Don't be hesitant to come here to get weapon i...,Đừng ngại! Lại đây lấy binh khí đi. ở đây có n...,0,0,0,0,0,0
1,1,2,SN_NPC_CH_ARMOR_BS,먼 길을 떠나려거든 준비를 단단히 하세요. 특히 자신의 몸을 보호하는 것에 아까워 ...,0,0,0,0,0,Brace yourself up for a long way. You shouldn'...,Hãy chuẩn bị để đi một chuyến dài. Phải đặc bi...,0,0,0,0,0,0
2,1,3,SN_NPC_CH_POTION_BS,끊임없이 몰려드는 환자들 때문에 일손도 약도 모자를 지경이라네. 이러다가 사람들에게...,0,0,0,0,0,"With consistent patients swarming in, we are a...",Nhân lực và thuốc thang đang thiếu do bệnh nhâ...,0,0,0,0,0,0
3,1,4,SN_NPC_CH_ACCESSORY_BS,어서오세요. 예쁜 장신구 구경하세요. 손님에게 꼭 맞는 맞춤형 장신구가 여기 있어요...,0,0,0,0,0,Do come in. Look around pretty jewlery. The pe...,Xin chào quý khách. Mời quý khách xem hàng. vừ...,0,0,0,0,0,0
4,1,5,SN_NPC_CH_HORSE_BS,어..어서..오..세..요.. 말… 을 사시… 려굽쇼? 헤..헤.헤.. 좋은..말....,0,0,0,0,0,Can..I..help.. you? Are..you..going..to.. buy ...,Quý khách có muốn mua ngựa không? He…he…he… ở ...,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7103,1,895,SN_QNO_TQ_GENERALORDER_1,손현의 도움요청,0,0,0,0,0,Sonhyeon's Plea,Yêu cầu giúp đỡ của Tôn Huyền,0,0,0,0,0,0
7104,1,896,SN_QNO_TQ_GENERALORDER_2,촉수 수집,0,0,0,0,0,Colleting the Tentacles,Thu thập xúc tu,0,0,0,0,0,0
7105,1,897,SN_QNO_TQ_GENERALORDER_3,사린족 토벌,0,0,0,0,0,Campaign against the Tomb Snake,Dàn trận chống Xà Lân,0,0,0,0,0,0
7106,1,898,SN_QNO_TQ_GENERALORDER_4,위험제거,0,0,0,0,0,Eliminate the Threat,Loại trừ hiểm nguy,0,0,0,0,0,0
